In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import wfdb
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset



In [ ]:
!pip3 install wfdb

In [ ]:


# In[3]:


class EcgDataset(Dataset):
    
    def __init__(self, data,labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]


def labels_nametonum():

    raw_labels = [270492004,
                  164889003,
                  164890007,
                  426627000,
                  713427006,
                  713426002,
                  445118002,
                  39732003,
                  164909002,
                  251146004,
                  698252002,
                  10370003,
                  284470004,
                  427172004,
                  164947007,
                  111975006,
                  164917005,
                  47665007,
                  59118001,
                  427393009,
                  426177001,
                  426783006,
                  427084000,
                  63593006,
                  164934002,
                  59931005,
                  17338001
                  ]

    raw_labels_lower = raw_labels #[l.lower() for l in raw_labels]

    raw_labels = dict(zip(raw_labels_lower, list(range(len(raw_labels_lower)))))

    return raw_labels


mapped_labels = labels_nametonum()

def map_labels(pre_labels):

    labels = []
    label_mapping = labels_nametonum()
    for name in pre_labels:
        # if name =="healthy control":
        #     labels.append(label_mapping["healthy controls"])
        # if name == 'healthy control':
        #     pass
        #     #labels.append(label_mapping['healthy controls'])
        # elif name in "cardiomyopathy/heart failure":
        #     labels.append(label_mapping["cardiomyopathy/heart failure"])
        if name in label_mapping:
            labels.append(label_mapping[name])
        # else:
        #     print("ERROR: label \"%s\" not in mapping"%name)
        #     labels.append(200)
            
        
    return np.array(labels)

def map_label(raw_label):
        if raw_label in mapped_labels:
            return mapped_labels[raw_label]
            
        else:
            return 200        
        

In [ ]:

# In[45]:
class Preprocess:

    def __init__(self, path_to_dataset, eval=False):
        data = []
        labels = []

        #val_data? #test data?
        #val_data = np.array([])
        #test_data = np.array([])
        
        files =[]
        for dir in os.listdir(path1):
            files.append(dir.split('.')[0])
        patients = list(set(files))

        for p in patients:
            path_to_file = path_to_dataset+'/'+p
            if p[0].lower() != 'e':
              signal = wfdb.io.rdsamp(path_to_dataset+'/'+p)
            #data = np.concatenate((data,np.array(signal[0])), axis = 0)
            
              if(not(eval)):
                  label = self.find_label(signal[1])
                  print("lab",label)
                  if label != 'error':
                    extracted_label = map_label(label) #extract label and map
                    if extracted_label !=200:
                      signal_data = np.array(signal[0])
                      labels.append(extracted_label)
                      data.append(signal_data)
                      print(extracted_label)
                    else:
                      print("label is unscored")

                  # mapped_labels = map_labels(labels, signal_data)
              else:
                  data.append(np.array(signal[0]))

            
        self.data = np.array(data, dtype = object)
        if(not(eval)):
            self.labels = labels

    def get_data(self):
        return self.data, self.labels
      	 

    # def find_label(self, md):
    #     metadata_keys = list(md.keys())
    #     metadata_keys_lower = [k.lower() for k in metadata_keys]
    #     if 'comments' in metadata_keys_lower:
    #         comment_index = metadata_keys.index('comments')
    #         metadata_comments = md[metadata_keys[comment_index]]
    #         metadata_comments_lower = [k.lower() for k in metadata_comments]
    #         label = ''
    #         for i, stri in enumerate(metadata_comments_lower):
    #             if 'reason for admission:' in stri:
    #                 label = stri.split(':')[1].strip()
    #         if label =='':
    #             print('reason for admission not in metadata')
    #         return label

    #     else:
    #         print("comments not in metadata")
    #         return 'error'

    def find_label(self, md):
        metadata_keys = list(md.keys())
        metadata_keys_lower = [k.lower() for k in metadata_keys]
        if 'comments' in metadata_keys_lower:
            comment_index = metadata_keys_lower.index('comments')
            metadata_comments = md[metadata_keys[comment_index]]
            metadata_comments_lower = [k.lower() for k in metadata_comments]
            label = ''
            for i, stri in enumerate(metadata_comments_lower):
                if 'dx:' in stri:
                    label = stri.split(':')[1].strip()
                    if label =='' or ',' in label:
                        print('Dx: not in metadata or two labels found --', label)
                        return 'error'

                    return int(label)

        else:
            print("comments not in metadata")
            return 'error'

In [ ]:
# !pip3 install wfdb


In [ ]:
!unzip "/content/drive/MyDrive/DLProjectPTBdataset/physionet2020_data.zip"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path1 = "zfsauton/data/public/vsanil/physionet/physionet_challenge_2020/train_data"
batch_size = 128
procdata = Preprocess(path1)
#np.save(procdata, procdata.npy )

# #train data
# dataset_train = EcgDataset(procdata.data, procdata.labels)
# train_args = dict(shuffle = True, batch_size = batch_size, num_workers=0)
# train_dataloader = DataLoader(dataset_train, shuffle=True, batch_size=128, num_workers=0)

print("data preprocessed")

Streaming output truncated to the last 5000 lines.
lab error
Dx: not in metadata or two labels found -- 426783006,63593006
lab error
Dx: not in metadata or two labels found -- 164861001,164873001,284470004,426783006
lab error
lab 426783006
21
lab 164867002
label is unscored
Dx: not in metadata or two labels found -- 426177001,426783006
lab error
Dx: not in metadata or two labels found -- 426783006,427084000
lab error
Dx: not in metadata or two labels found -- 164861001,164873001,39732003,426783006
lab error
lab 164861001
label is unscored
lab 426783006
21
Dx: not in metadata or two labels found -- 164861001,164873001,164889003,164951009
lab error
Dx: not in metadata or two labels found -- 413844008,164867002,164865005,164889003
lab error
lab 426783006
21
lab 164889003
1
Dx: not in metadata or two labels found -- 164865005,164951009,426783006
lab error
Dx: not in metadata or two labels found -- 164865005,164951009,426783006
lab error
Dx: not in metadata or two labels found -- 164951009,

In [ ]:
from torch.nn.utils.rnn import pad_sequence
def collate_batch(batch):
   
  label_list, text_list, = [], []
   
  for (_text,_label) in batch:
    label_list.append(_label)
    text_list.append(torch.tensor(_text))
   
  label_list = torch.tensor(label_list, dtype=torch.int64)
   
  text_list = pad_sequence(text_list, batch_first=True, padding_value=0)
   
  return text_list.to(device),label_list.to(device),

In [ ]:
#train data
dataset_train = EcgDataset(procdata.data, procdata.labels)
train_args = dict(shuffle = True, batch_size = batch_size, num_workers=0)
train_dataloader = DataLoader(dataset_train, shuffle=True, batch_size=128, num_workers=0,collate_fn=collate_batch)

In [ ]:
d = {}
for i in procdata.labels:
  if i not in d:
    d[i] = 1
  else: 
    d[i] +=1

In [ ]:
class MyConv1dPadSame(nn.Module):
    """
    extend nn.Conv1d to support SAME padding
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super(MyConv1dPadSame, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.conv = torch.nn.Conv1d(
            in_channels=self.in_channels, 
            out_channels=self.out_channels, 
            kernel_size=self.kernel_size, 
            stride=self.stride, 
            groups=self.groups)

    def forward(self, x):
        
        net = x
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.conv(net)
        return net
        
class MyMaxPool1dPadSame(nn.Module):
    """
    extend nn.MaxPool1d to support SAME padding
    """
    def __init__(self, kernel_size):
        super(MyMaxPool1dPadSame, self).__init__()
        self.kernel_size = kernel_size
        self.stride = 1
        self.max_pool = torch.nn.MaxPool1d(kernel_size=self.kernel_size)

    def forward(self, x):
        
        net = x
        
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.max_pool(net)
        
        return net
    
class BasicBlock(nn.Module):
    """
    ResNet Basic Block
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups, downsample, use_bn, use_do, is_first_block=False):
        super(BasicBlock, self).__init__()
        
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.stride = stride
        self.groups = groups
        self.downsample = downsample
        if self.downsample:
            self.stride = stride
        else:
            self.stride = 1
        self.is_first_block = is_first_block
        self.use_bn = use_bn
        self.use_do = use_do

        # the first conv
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.relu1 = nn.ReLU()
        self.do1 = nn.Dropout(p=0.5)
        self.conv1 = MyConv1dPadSame(
            in_channels=in_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=self.stride,
            groups=self.groups)

        # the second conv
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.do2 = nn.Dropout(p=0.5)
        self.conv2 = MyConv1dPadSame(
            in_channels=out_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=1,
            groups=self.groups)
                
        self.max_pool = MyMaxPool1dPadSame(kernel_size=self.stride)

    def forward(self, x):
        
        identity = x
        
        # the first conv
        out = x
        if not self.is_first_block:
            if self.use_bn:
                out = self.bn1(out)
            out = self.relu1(out)
            if self.use_do:
                out = self.do1(out)
        out = self.conv1(out)
        
        # the second conv
        if self.use_bn:
            out = self.bn2(out)
        out = self.relu2(out)
        if self.use_do:
            out = self.do2(out)
        out = self.conv2(out)
        
        # if downsample, also downsample identity
        if self.downsample:
            identity = self.max_pool(identity)
            
        # if expand channel, also pad zeros to identity
        if self.out_channels != self.in_channels:
            identity = identity.transpose(-1,-2)
            ch1 = (self.out_channels-self.in_channels)//2
            ch2 = self.out_channels-self.in_channels-ch1
            identity = F.pad(identity, (ch1, ch2), "constant", 0)
            identity = identity.transpose(-1,-2)
        
        # shortcut
        out += identity

        return out
    
class ResNet1D(nn.Module):
    """
    
    Input:
        X: (n_samples, n_channel, n_length)
        Y: (n_samples)
        
    Output:
        out: (n_samples)
        
    Pararmetes:
        in_channels: dim of input, the same as n_channel
        base_filters: number of filters in the first several Conv layer, it will double at every 4 layers
        kernel_size: width of kernel
        stride: stride of kernel moving
        groups: set larget to 1 as ResNeXt
        n_block: number of blocks
        n_classes: number of classes
        
    """

    def __init__(self, in_channels, base_filters, kernel_size, stride, groups, n_block, n_classes, downsample_gap=2, increasefilter_gap=4, use_bn=True, use_do=True, verbose=False):
        super(ResNet1D, self).__init__()
        
        self.verbose = verbose
        self.n_block = n_block
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.use_bn = use_bn
        self.use_do = use_do

        self.downsample_gap = downsample_gap # 2 for base model
        self.increasefilter_gap = increasefilter_gap # 4 for base model

        # first block
        self.first_block_conv = MyConv1dPadSame(in_channels=in_channels, out_channels=base_filters, kernel_size=self.kernel_size, stride=1)
        self.first_block_bn = nn.BatchNorm1d(base_filters)
        self.first_block_relu = nn.ReLU()
        out_channels = base_filters
                
        # residual blocks
        self.basicblock_list = nn.ModuleList()
        for i_block in range(self.n_block):
            # is_first_block
            if i_block == 0:
                is_first_block = True
            else:
                is_first_block = False
            # downsample at every self.downsample_gap blocks
            if i_block % self.downsample_gap == 1:
                downsample = True
            else:
                downsample = False
            # in_channels and out_channels
            if is_first_block:
                in_channels = base_filters
                out_channels = in_channels
            else:
                # increase filters at every self.increasefilter_gap blocks
                in_channels = int(base_filters*2**((i_block-1)//self.increasefilter_gap))
                if (i_block % self.increasefilter_gap == 0) and (i_block != 0):
                    out_channels = in_channels * 2
                else:
                    out_channels = in_channels
            
            tmp_block = BasicBlock(
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=self.kernel_size, 
                stride = self.stride, 
                groups = self.groups, 
                downsample=downsample, 
                use_bn = self.use_bn, 
                use_do = self.use_do, 
                is_first_block=is_first_block)
            self.basicblock_list.append(tmp_block)

        # final prediction
        self.final_bn = nn.BatchNorm1d(out_channels)
        self.final_relu = nn.ReLU(inplace=True)
        # self.do = nn.Dropout(p=0.5)
        self.dense = nn.Linear(out_channels, n_classes)
        # self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        out = x
        
        # first conv
        if self.verbose:
            print('input shape', out.shape)
        out = self.first_block_conv(out)
        if self.verbose:
            print('after first conv', out.shape)
        if self.use_bn:
            out = self.first_block_bn(out)
        out = self.first_block_relu(out)
        
        # residual blocks, every block has two conv
        for i_block in range(self.n_block):
            net = self.basicblock_list[i_block]
            if self.verbose:
                print('i_block: {0}, in_channels: {1}, out_channels: {2}, downsample: {3}'.format(i_block, net.in_channels, net.out_channels, net.downsample))
            out = net(out)
            if self.verbose:
                print(out.shape)

        # final prediction
        if self.use_bn:
            out = self.final_bn(out)
        out = self.final_relu(out)
        out = out.mean(-1)
        if self.verbose:
            print('final pooling', out.shape)
        # out = self.do(out)
        out = self.dense(out)
        if self.verbose:
            print('dense', out.shape)
        # out = self.softmax(out)
        if self.verbose:
            print('softmax', out.shape)
        
        return out    

In [ ]:
d

{0: 4,
 1: 40,
 2: 40,
 3: 1,
 4: 4,
 5: 13,
 6: 9,
 8: 13,
 10: 2,
 11: 273,
 12: 2,
 13: 12,
 15: 1,
 21: 6512,
 22: 2,
 23: 1,
 24: 32}

In [ ]:
procdata.data.shape, len(procdata.labels)

((6961,), 6961)

In [ ]:
meta = wfdb.io.rdsamp('zfsauton/data/public/vsanil/physionet/physionet_challenge_2020/train_data/S0226')

In [ ]:
meta[1]['comments']

['Age: 68',
 'Sex: male',
 'Dx: 164865005',
 'Rx: Unknown',
 'Hx: Unknown',
 'Sx: Unknown']

In [ ]:
pd.to_df()

# RESNET-34 **1D**

In [ ]:
# Resnet 34 architecture  with dropout layer before linear layer and no max pooling layer. 
# Helped retain features from images and trained model better. 
# Kernel size 3x3 and Stride 1 was used since size of images are 64x64 (which is pretty small)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=1,
                     padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=25, feat_dim = 2):
        super().__init__()
        
        self.inplanes = 64

        self.conv1 = nn.Conv1d(12, self.inplanes, kernel_size=3, stride=1,
                               bias=False)
        self.bn1 = nn.BatchNorm1d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.dropout2 = nn.Dropout(p=0.2)
        self.fc = nn.Linear(512 , num_classes)

        self.embedding_linear = nn.Linear(512, feat_dim)
        self.embedding_relu = nn.ReLU(inplace=True)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None  
   
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes, 1, stride, bias=False),
                nn.BatchNorm1d(planes),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        
        self.inplanes = planes
        
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x, return_embedding=False):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)          
        x = self.layer2(x)          
        x = self.layer3(x)          
        x = self.layer4(x)          

        x = self.avgpool(x)
        embedding = torch.flatten(x, 1)
        x = self.dropout2(embedding)
        output = self.fc(x)

        if return_embedding:
          return embedding ,output
        else:
          return output

In [ ]:
# Number of classes is 4000 for our dataset

def resnet34():
    layers=[3, 4, 6, 3]
    model = ResNet(BasicBlock, layers, num_classes=25)
    return model

network=resnet34()

In [ ]:
# Define hyperparameters

numEpochs = 10
in_features = 3 # RGB channels

learningRate = 0.1
weightDecay = 5e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = network.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

In [ ]:
# # make model
# device_str = "cuda"
# device = torch.device(device_str if torch.cuda.is_available() else "cpu")
# kernel_size = 16
# stride = 2
# n_block = 48
# downsample_gap = 6
# increasefilter_gap = 12
# model = ResNet1D(
#     in_channels=12, 
#     base_filters=128, # 64 for ResNet1D, 352 for ResNeXt1D
#     kernel_size=kernel_size, 
#     stride=stride, 
#     groups=1, 
#     n_block=n_block, 
#     n_classes=4, 
#     downsample_gap=downsample_gap, 
#     increasefilter_gap=increasefilter_gap, 
#     use_do=True)
# model.to(device)

In [ ]:
# optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
# criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Train and Validate Model

for epoch in range(numEpochs):
    
    # Train
    network.train()
    avg_loss = 0.0
    for batch_num, (x, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        x, y = x.type(torch.FloatTensor),y.type(torch.FloatTensor)
        x, y = x.to(device), y.to(device)

        feature, outputs = network(x.permute(0,2,1), return_embedding=True)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()

        if batch_num % 50 == 49:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/50))
            avg_loss = 0.0

    

    # Validate
    network.eval()
    num_correct = 0
    for batch_num, (x, y) in enumerate(val_dataloader):
        x, y = x.to(device), y.to(device)
        outputs = network(x)
        val_loss = criterion(outputs, y.long())
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    scheduler.step(val_loss)

#Save model after every epoch in order to choose the best model with highest accuracy
    torch.save({
        'epoch': epoch,
        'model_state_dict': network.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, f'/content/drive/My Drive/sramu_model2_epoch_'+str(int(epoch)))
 
    print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(val_dataset)))


RuntimeError: ignored